In [6]:
import math
import pandas as pd
import numpy as np
from scipy import constants
from astropy import constants as astro_const
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt



import black_holes as bh

In [7]:
pd.set_option('display.float_format', '{:.2e}'.format)

# Step 1: Define the Black Hole's Temperature

First, you need the formula for the **Hawking temperature ($T$)**, which is inversely proportional to the black hole's mass ($M$). For a non-rotating, uncharged Schwarzschild black hole, the equation is:

$$T = \frac{\hbar c^3}{8\pi G M k_B}$$

Where:
* $\hbar$ is the reduced Planck constant
* $c$ is the speed of light
* $G$ is the gravitational constant
* $k_B$ is the Boltzmann constant


In [8]:
print(bh.hawking_temperature(6e30))

2.0448344509900284e-08


# Step 2: Calculate the Mass Loss Rate

The black hole's mass loss rate is determined by the power it radiates as Hawking radiation. This power can be calculated using the **Stefan-Boltzmann law**, which relates an object's temperature to the power it emits. The power ($P$) is the energy radiated per unit time, and by $E=mc^2$, this corresponds to a mass loss rate ($dM/dt$):

$$\frac{dM}{dt} = - \frac{P}{c^2} = - \frac{A \sigma T^4}{c^2}$$

Where:
* $A$ is the surface area of the event horizon, $A = 4\pi r_s^2 = 4\pi (2GM/c^2)^2 = 16\pi G^2M^2/c^4$
* $\sigma$ is the Stefan-Boltzmann constant
* $T$ is the Hawking temperature calculated in Step 1

By substituting the expression for $T$ and $A$ into the mass loss rate equation, you get a final formula that relates the mass loss rate to the black hole's current mass:

$$\frac{dM}{dt} = - \frac{\hbar c^4}{15360 \pi G^2 M^2}$$

Notice that the mass loss rate is **inversely proportional to the square of the mass**. This is the key insight: as a black hole loses mass, it gets hotter and evaporates at a faster and faster rate. 


# Step 3: Simulate the Evaporation

To simulate the evaporation from an initial mass ($M_i$) to zero, you must solve the differential equation for the mass loss rate. Since the rate of change depends on the current mass, a simple numerical integration is the most common approach.

Starting with an initial mass $M_i$ at $t=0$, you can use a small time step $\Delta t$ to update the mass:

$$M(t+\Delta t) = M(t) + \frac{dM}{dt} \times \Delta t$$

You repeat this process for many small steps. At each step, you:
1.  Calculate the current temperature using the current mass.
2.  Calculate the current mass loss rate.
3.  Update the mass for the next time step.

This iterative process will show the black hole's mass and temperature evolve over time. The mass will decrease, slowly at first, and then accelerate toward zero, with the temperature doing the exact opposite.

The total time for a black hole to evaporate completely, from an initial mass $M_i$, is given by integrating the mass loss rate equation:

$$\tau \approx \left(\frac{M_i}{M_{\odot}}\right)^3 \times 10^{66} \text{ years}$$

This formula shows that a solar-mass black hole will take an incomprehensibly long time to evaporate, far longer than the current age of the universe. Only much smaller, "primordial" black holes could have evaporated by now.


In [9]:
masses_kg = [
    1e15, # PBH
    10 * astro_const.M_sun.value, # Stellar BH
    1e9 * astro_const.M_sun.value, # Supermassive BH
]

In [ ]:
# Calculate the Hawking temperature for each mass
hawking_temperatures = [bh.hawking_temperature(m) for m in masses_kg]

# Create a DataFrame to display the results in a table
df = pd.DataFrame({
    'Mass (kg)': masses_kg,
    'Hawking Temperature (K)': hawking_temperatures
})

# Display the table
print(df)


m0 = masses_kg[0]
t_span = [0, 1e15]  # Time interval [start, end]


time_step_for_output = 100
t_eval_points = np.arange(t_span[0], t_span[1] + time_step_for_output, time_step_for_output)

solution = solve_ivp(
    fun=bh.dM_dt, 
    t_span=t_span, 
    t_eval=t_eval_points, 
    y0=[m0], 
    dense_output=True, # Allows for continuous solution evaluation
 )


# Create a smooth time array for plotting
t_plot = np.linspace(t_span[0], t_span[1], 1000)
y_solve_ivp = solution.sol(t_plot)[0] # Extract the solution values

plt.figure(figsize=(8, 4))
plt.plot(t_plot, y_solve_ivp, label='solve_ivp (Runge-Kutta)', color='red')
plt.title(r'Accurate Numerical Solution of $\frac{dM}{dt} = - \frac{\hbar c^4}{15360 \pi G^2 M^2}$')
plt.xlabel('Time (t)')
plt.ylabel('M(t)')
plt.legend()
plt.grid(True)
plt.show()


   Mass (kg)  Hawking Temperature (K)
0   1.00e+15                 1.23e+08
1   1.99e+31                 6.17e-09
2   1.99e+39                 6.17e-17
